In [ ]:
import pymongo
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

# Creating the Connections

In [ ]:
# create the connection to MongoDB
# create an instance of the database and collections

conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

db = client.australia_fire_db
bushfiresbyState = db.bushfiresbyState
historicalFires = db.historicalFires
aus2019_2020 = db.aus2019_2020

In [ ]:
# urls to scrape

byStateurl = 'https://en.wikipedia.org/wiki/2019%E2%80%9320_Australian_bushfire_season'
historicalurl = 'https://en.wikipedia.org/wiki/List_of_major_bushfires_in_Australia'
aus2019_2020url = 'https://en.m.wikipedia.org/wiki/List_of_fires_and_impacts_of_the_2019-20_Australian_bushfire_season'

In [ ]:
# request the html using beautiful soup

historical_response = requests.get(historicalurl)
bystate_response = requests.get(byStateurl)
aus2019_2020_response = requests.get(aus2019_2020url)

In [ ]:
# parse the html text

h_soup = BeautifulSoup(historical_response.text, 'html.parser')
bs_soup = BeautifulSoup(bystate_response.text, 'html.parser')
a_soup = BeautifulSoup(aus2019_2020_response.text, 'html.parser')

# Historical Bushfire Scrapping

In [ ]:
# the table headers are within the table body in the first two rows

h_table = h_soup.find('table', class_="wikitable")
h_table_headers = h_soup.find_all("tr")[0:2]

In [ ]:
# scrape the table headers and append them to the headers list

h_headers = []
for table_header in h_table_headers:
    try:
        value = table_header.text
#         value = value.split("\n")
        h_headers.append(value)
        
    except AttributeError as e:
        print(e)
        
print(h_headers)
print(len(h_headers))

In [ ]:
# manually create the headers list, because the scraped table headers was too complicated
# if there is time, create logic to put the two header rows into one similar to the list below

h_headers = ['Date', 'Name', 'State(s)/territories', 'AreaBurned(ha)', 'AreaBurned(acres)', 'Fatalities', 'PropertiesDamaged(HomesDestroyed)']

In [ ]:
# find objects to scrape
# the table data is after the headers which is in the first two rows

h_table = h_soup.find('table', class_="wikitable")
h_table_body = h_table.find("tbody")
h_table_row = h_table_body.find_all('tr')[2:]

In [ ]:
# scrape data and create a list of lists for each row of data

h_data = []

for row in h_table_row:
    
    datarow = []
    
    table_data = row.find_all('td')
    
    for tdata in table_data:
        try:
            value = tdata.text
    #             value.split("\n\n")
            value = value.replace("\n", "")
            datarow.append(value)

        except AttributeError as e:
            print(e)
            
    h_data.append(datarow)
        
# print(h_data)

In [ ]:
# convert scrapped data into a dataframe

h_df = pd.DataFrame(h_data)
h_df

In [ ]:
# keep desired rows

h_drop_rows_df = h_df[[0,1,2,3,4,5,6]]
h_drop_rows_df

In [ ]:
# Add column headers

h_drop_rows_df.columns = h_headers
h_drop_rows_df

In [ ]:
# replace Nil with 0 values

h_nil_df = h_drop_rows_df.replace('Nil', '0')
h_nil_df.head(10)

In [ ]:
# remove commas from numbers
# replace unwanted values as NaNs
# replace range data into an average value
# remove citations found within []

h_tonumeric_df = h_nil_df.copy()
h_tonumeric_df['AreaBurned(ha)'] = h_nil_df['AreaBurned(ha)'].str.replace(',','')
h_tonumeric_df['AreaBurned(acres)'] = h_nil_df['AreaBurned(acres)'].str.replace(',','')
h_tonumeric_df['PropertiesDamaged(HomesDestroyed)'] = h_nil_df['PropertiesDamaged(HomesDestroyed)'].str.replace(',','')

columnstoedit = ['AreaBurned(ha)','AreaBurned(acres)','Fatalities','PropertiesDamaged(HomesDestroyed)']

for i in range(len(h_tonumeric_df['Date'])):
    for column in columnstoedit:
        
        if ((h_tonumeric_df[column][i] == '') and (h_tonumeric_df[column][i] != '0')) or (h_tonumeric_df[column][i] == 'unknown') :
            h_tonumeric_df[column][i] = 'NaN'

        if 'approx. ' in str(h_tonumeric_df[column][i]):
            h_tonumeric_df[column][i] = h_tonumeric_df[column][i].replace('approx. ', '')
            
        if 'than ' in str(h_tonumeric_df[column][i]):
            h_tonumeric_df[column][i] = h_tonumeric_df[column][i].split(' ')[-1]

        if len(str(h_tonumeric_df[column][i]).split('–')) == 2:
            splitvalues = str(h_tonumeric_df[column][i]).split('–')
            h_tonumeric_df[column][i] = np.mean([int(splitvalues[0]), int(splitvalues[1])])

        if len(str(h_tonumeric_df[column][i]).split('[')) >= 2:
            h_tonumeric_df[column][i] = str(h_tonumeric_df[column][i]).split('[')[0]
        

h_tonumeric_df

In [ ]:
# pull out the year the fire took place, if over two years, grab the beginning year

h_year_df = h_tonumeric_df.copy()
h_year_df['Year'] = ''
for i in range(len(h_year_df['Date'])):
    
    resultslist = []
    results = str(h_year_df['Date'][i]).split(' ')

    for result in results:
        try:
            value = int(result)
            if len(str(value)) == 4:
                resultslist.append(value)
                year = np.min(resultslist)
                h_year_df['Year'][i] = year

        except:
            year = 'NaN'
            h_year_df['Year'][i] = year
    
h_year_df[:50]


In [ ]:
# remove nan values and empty values and convert them to integers

h_casting_df = h_year_df.copy()

h_casting_df = h_casting_df[h_casting_df['AreaBurned(ha)'] != 'NaN']
h_casting_df = h_casting_df[h_casting_df['PropertiesDamaged(HomesDestroyed)'] != 'NaN']
h_casting_df = h_casting_df[h_casting_df['PropertiesDamaged(HomesDestroyed)'] != '']
h_casting_df = h_casting_df[h_casting_df['Year'] != 'NaN']
h_casting_df = h_casting_df.astype(
    {
        'AreaBurned(ha)':'int',
        'AreaBurned(acres)': 'int',
        'Fatalities':'int',
        'PropertiesDamaged(HomesDestroyed)':'int',
        'Year': 'int'
    })
# h_casting_df['Year'] = pd.to_datetime(h_casting_df['Year'], format = '%Y')
h_casting_df.dtypes
h_casting_df

In [ ]:
# transform the dataframe into a dictionary so that we can feed it to MongoDB
h_dict = h_casting_df.to_dict('records')
h_dict

In [ ]:
# insert records into the MongoDB collection histroicalFires

if (historicalFires.count() == 0):
    historicalFires.insert(h_dict)
    
else:
    print("Data already exists")

# Scrapping Data from 2019-2020 by State

In [ ]:
# scrape the table of interest

bs_table = bs_soup.find('table', class_='sortable')
print(bs_table.prettify())

In [ ]:
# scrape the column headers

bs_theaders = bs_soup.find('table', class_='sortable').find_all('tr')[0:2]


bs_headers = []
for table_header in bs_theaders:
    try:
        value = table_header.text
        value = value.split("\n\n")
        bs_headers.append(value)
        
    except AttributeError as e:
        print(e)
        
print(bs_headers)
print(len(bs_headers))

In [ ]:
# manually create the column headers

bs_theaders = ['State/Territory', 'Fatalities', 'Homeslost', 'Area(estimated)(ha)', 'Area(estimated)(acres)', 'Notes']

In [ ]:
# scrape data and create a list of lists for each row of data

bs_table_row = bs_soup.find('table', class_='sortable').find_all('tr')[2:]

bs_data = []

for row in bs_table_row:
    
    datarow = []
    
    table_header = row.find('th').text
    table_header = table_header.replace('\n','')
    datarow.append(table_header)
    table_data = row.find_all('td')
    
    for tdata in table_data:
        try:
            value = tdata.text
    #             value.split("\n\n")
            value = value.replace("\n", "")
            datarow.append(value)

        except AttributeError as e:
            print(e)
            
    bs_data.append(datarow)
        
print(bs_data)

In [ ]:
# convert scrapped data into a dataframe

bs_df = pd.DataFrame(bs_data)
bs_df

In [ ]:
# Add column headers

bs_df.columns = bs_theaders
bs_df

In [ ]:
# remove any unwated characters from the columns

bs_dropchar_df = bs_df.drop('Notes', axis=1)

columnstoedit = ['Homeslost', 'Area(estimated)(ha)', 'Area(estimated)(acres)']

bs_dropchar_df["Homeslost"] = bs_dropchar_df["Homeslost"].replace({
    "3,500+": "3500"
})

for column in columnstoedit:
    for i in range(len(bs_dropchar_df[column])):
        try:
            bs_dropchar_df[column][i] = str(bs_dropchar_df[column][i]).replace(',','')
            bs_dropchar_dfr_df[column][i] = str(bs_dropchar_df[column][i]).replace('+', '')
        except:
            pass

bs_dropchar_df

In [ ]:
# case the columns of interest

bs_cast_df = bs_dropchar_df.copy()

bs_cast_df = bs_cast_df.astype({
    'Fatalities': 'int',
    'Homeslost': 'int',
    'Area(estimated)(ha)': 'int',
    'Area(estimated)(acres)': 'int'
})

bs_cast_df.dtypes

In [ ]:
# create a dictionary of our dataframe

bs_dict = bs_cast_df.to_dict('records')
bs_dict

In [ ]:
# put the data into mongodb

if (bushfiresbyState.count() == 0):
    bushfiresbyState.insert(bs_dict)
    
else:
    print("Data already exists")

# Australian 2019 - 2020 Bushfire Impact

In [ ]:
# grab the table and the table rows from the site of interest

a_table = a_soup.find('table', class_="wikitable")
a_table_body = a_table.find_all("tr")

In [ ]:
# scrape all of the table rows (includeing the headers)

a_tdata = []
for table_header in a_table_body:
    data_row = []
    
    t_headers = table_header.find_all("th")
    for header in t_headers:
        try:
            value = header.text
            value = value.replace("\n", "")
            data_row.append(value)

        except AttributeError as e:
            print(e)

    t_data = table_header.find_all("td")
    for data in t_data:
        try:
            value = data.text
            value = value.replace("\n", "")
            data_row.append(value)

        except AttributeError as e:
            print(e)
            
    a_tdata.append(data_row)
        
print(a_tdata)

In [ ]:
# create a dataframe from the scrapped data

a_df = pd.DataFrame(a_tdata)
a_df

In [ ]:
# use the first row as the column header

a_setheaders_df = a_df.copy()
a_setheaders_df.columns = a_setheaders_df.iloc[0]
a_setheaders_df = a_setheaders_df.drop(a_setheaders_df.index[0])
a_setheaders_df

In [ ]:
# remove any unwatnted columns and rename some column headers

a_editcols_df = a_setheaders_df.copy()
a_editcols_df = a_editcols_df.drop(['Start Date', 'Contained / Cease Date', 'Comments'], axis=1)
a_editcols_df = a_editcols_df.rename({'Area impacted': 'AreaImpacted(ha)'}, axis=1)
a_editcols_df.head()

In [ ]:
# remove any unwanted characters from columns

a_parsedata_df = a_editcols_df.copy()

for i in range(len(a_parsedata_df['Local Government Area(s)'])):
    try:

        a_parsedata_df['Local Government Area(s)'][i] = str(a_parsedata_df['Local Government Area(s)'][i]).split('[')[0]
    except:
        pass
        
for i in range(len(a_parsedata_df['AreaImpacted(ha)'])+1):
    try:
        value = str(a_parsedata_df['AreaImpacted(ha)'][i]).split(' ')[0]
        value = value.replace(',','')
        a_parsedata_df['AreaImpacted(ha)'][i] = value
    except:
        pass
    
a_parsedata_df.head()

In [ ]:
# drop any rows that don't have values in the area impacted column

a_droprows_df = a_parsedata_df.copy()

a_droprows_df = a_droprows_df[a_droprows_df['AreaImpacted(ha)']!='']
a_droprows_df.head()

In [ ]:
# cast the columns of interest

a_cast_df = a_droprows_df.copy()
a_cast_df = a_cast_df.astype({
    'AreaImpacted(ha)': 'int'
})
a_cast_df.dtypes

In [ ]:
# put the final dataframe into a dictionary

a_dict = a_cast_df.to_dict('record')
a_dict

In [ ]:
# insert records into the MongoDB collection histroicalFires

if (aus2019_2020.count() == 0):
    aus2019_2020.insert(a_dict)
    
else:
    print("Data already exists")